In [26]:
import pandas as pd 
import numpy as np 
import csv


In [40]:
df = pd.read_csv ("Data/factory.csv", header=0, delim_whitespace=True)

In [42]:
df.head()

,time_batch,factory_average_kw,work_station_average_kw,temp_CNC_1,humidity_CNC_1,temp_CNC_2,humidity_CNC_2,temp_CNC_3,humidity_CNC_3,temp_CNC_4,...,temp_CNC_9,humidity_CNC_9,temp_outside,press_mm_hg_outside,humidity_outside,windspeed_outside,visibility_outside,dewpoint_outside,temp_CNC_10,humidity_CNC_10
11-01-2016,17:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
11-01-2016,17:10,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
11-01-2016,17:20,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
11-01-2016,17:30,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0,45.410390,45.410390
11-01-2016,17:40,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097
